# Greedy Search Decoding algorithm

연속적인 모델 출력에서 이산적인 토큰을 얻는 가장 간단한 방법은 각 스템마다 확률이 가장 높은 토큰을 탐욕적으로 선택하는거

In [ ]:
#그리디 서치 방법을 알아보기 위해 언어 모델링 헤더를 가진 15억개 파라미터의 GPT-2 버전을 로드하겠습니다
import torch
from transformers import AutoTokenizer,AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

허깅페이스 트렌스포머는 GPT-2 같은 자기회귀 모델을 위해 generate() 함수를 제공하지만, 작동 방식을 이해하기 위해 직접 이 디코딩 메서드를 구현했습니다.
입력프롬프트로 Transformer are the를 사용하고 여덟 번의 타임스텝 동안 디코딩을 수행함.

각 타임스텝에서 프롬프트의 마지막 토큰에 대한 로짓을 선택하고 소프트맥스를 적용해 확률분포를 얻었습니다.

그다음 확률이 가장 높은 토큰을 다음토큰으로 선택하고 입력 시퀀스에 추가한 후에 이 과정을 다시 반복. 또 대안을 시각적으로 보여주기 위해 타임스텝마다 확률이 가장 높은 토큰 다섯개를 지정함

In [ ]:
import pandas as pd

input_txt = "Transformers are the"
input_ids  =tokenizer(input_txt,return_tensors ="pt")["input_ids"].to(device)
'''
{
  'input_ids': tensor([[101, 19081, 2024, 1996]]),
  'attention_mask': tensor([[1, 1, 1, 1]])
}
tokenizer의 반환값은 딕셔너리 형태로 반환한다.
["input_ids"]를 바로 붙이면 이 딕셔너리에서 input_ids 값만 꺼내서 쓰겠다는 거임

'''
iterations = []#리스트로 하는 이유는 나중에 iteration[1]처럼 각 단계별 결과를 비교하거나 시각화하기 쉽기 때문임


n_steps = 8
choices_per_step = 5

with torch.no_grad():
  for _ in range(n_steps):#변수를 _로 받아버리면 n_steps번 하지만, 인덱스값은 안쓰겠다는 뜻
    iteration = dict()#새 딕셔너리를 만듬 매 반복마다 결과(입력문장,각 후보 문장 토큰 등)를 여기에 저장하려는 용도
    iteration["input"] = tokenizer.decode(input_ids[0])#tokenizer.decode()는 ID를 사람이 읽을 수 있는 문장으로 변환
                                                      # input이라는 새로운 딕셔너리 키를 생성한거임 {"input" : "Transformer are the"}
                                                      #트랜스포머 모델은 항상 2차원으로 되어있음 나는 문장 1개만 넣으니깐 batch_size = 1
                                                      #그래서 input_ids는 실제로 tensor([[101,10981,2024,1996,3936]])
                                                      #따라서 input_ids 전체는 shape=(1,seq_len)
                                                      #input_ids[0] 의 shape = (seq_len,)
                                                      #그리고 decode()는 1차원 리스트 형태여야함
                                                      #모든 트렌스포머는 batch 처리가 기본임



    output = model(input_ids = input_ids)
    #첫번째 배치의 마지막 토큰의 로짓을 선택해 소프트맥스를 적용합니다
    next_token_logits = output.logits[0,-1,:]#문장1개니깐 첫번째 배치 선택하고 마지막 토큰을 꺼내고 그 위치에서 가능한 모든 단어의 logit값
                                             #지금까지의 문장을 보고 다음단어가 뭘까에 대한 원시 점수 (logit)을 가져옴

    next_token_probs  = torch.softmax(next_token_logits,dim=-1)#마지막차원 기준 softmax 적용
    sorted_ids = torch.argsort(next_token_probs,dim=-1,descending=True)#argsort() 확률을 정렬한 인덱스 순서를 반환
    #가장 높은 확률의 토큰을 저장합니다
    for choice_idx in range(choices_per_step):#상위 5개 단어만 추출
      token_id = sorted_ids[choice_idx]
      token_prob = next_token_probs[token_id].cpu().numpy()
      token_choice = (f"{tokenizer.decode(token_id)} ( {100 * token_prob : .2f}%)")
      iteration[f"Choice {choice_idx+1}"] = token_choice

    #예측한 다음 토큰을 입력에 추가합니다
    input_ids = torch.cat([input_ids,sorted_ids[None,0,None]],dim = -1)#그냥 sorted_ids[0]은 스칼라 라서 input_ids에 붙이려면 shape를 맞춰야함
                                                                      '''
                                                                      input_ids:  [[101, 1023, 2024, 1996, 1234]]
                                                                      new_token : [[345]]

                                                                      합치면 → [[101, 1023, 2024, 1996, 1234, 345]]
                                                                      '''

    iterations.append(iteration)


pd.DataFrame(iterations)

,input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,Transformers are the,most ( 8.53%),only ( 4.96%),best ( 4.65%),Transformers ( 4.37%),ultimate ( 2.16%)
1,Transformers are the most,popular ( 16.78%),powerful ( 5.37%),common ( 4.96%),famous ( 3.72%),successful ( 3.20%)
2,Transformers are the most popular,toy ( 10.63%),toys ( 7.23%),Transformers ( 6.60%),of ( 5.46%),and ( 3.76%)
3,Transformers are the most popular toy,line ( 34.38%),in ( 18.20%),of ( 11.71%),brand ( 6.10%),line ( 2.69%)
4,Transformers are the most popular toy line,in ( 46.29%),of ( 15.09%),", ( 4.94%)",on ( 4.40%),ever ( 2.72%)
5,Transformers are the most popular toy line in,the ( 65.99%),history ( 12.42%),America ( 6.91%),Japan ( 2.44%),North ( 1.40%)
6,Transformers are the most popular toy line in the,world ( 69.27%),United ( 4.55%),history ( 4.29%),US ( 4.23%),U ( 2.30%)
7,Transformers are the most popular toy line in ...,", ( 39.73%)",. ( 30.64%),and ( 9.87%),with ( 2.32%),today ( 1.74%)


In [ ]:
input_ids = tokenizer(input_txt,return_tensors = "pt")["input_ids"].to(device)#문자열 -> 토큰ID 텐서로 변환
                                                                              #예를들어 input_txt = "Transformers are the " 라면 이걸 tokenizer가 딕셔너리 형태로 바꿔줌
                                                                              #{
                                                                                 #"input_ids" : tensort([[101,10981,2024,1896,102]]),
                                                                                 #attention_mask : tensor([[1,1,1,1,1]])
                                                                              #}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Transformers are the most popular toy line in the world,


In [ ]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.\n\n"""


input_ids = tokenizer(input_txt,return_tensors = "pt")["input_ids"].to(device)
#input_ids2 = tokenizer(input_txt2,return_tensors = "pt")["input_ids"].to(device)
output_greedy =model.generate(input_ids,max_length = max_length,do_sample =False)#do_sample 매개변수는 토큰을 랜덤으로 뽑지말고 확률 높은 토큰만 뽑아라
                                                                                  # input_ids에 이어서 문장들을 생성하는 로직
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees.


The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.


The researchers were surprised to find that the unicorns were able


# Dim Search Decoding algorithm

빔 서치(Beam Search) 디코딩은 **그리디 서치(Greedy Search)**의 한계를 보완한 알고리즘이다.

그리디 서치는 매 단계에서 입력 토큰과 출력 토큰을 비교하여 가장 높은 확률을 가진 단어 하나만 선택하기 때문에,
탐색 경로가 단 하나뿐이다.
따라서 한 번 잘못된 토큰이 선택되면 이후 전체 문장이 비자연스럽게 이어지는 치명적인 오류가 발생할 수 있다.

반면, 빔 서치는 매 스텝마다 상위 **b개(빔 크기, beam size)**의 후보 토큰을 동시에 추적한다.
이렇게 하면 여러 개의 가능한 시퀀스를 병렬로 확장·평가할 수 있으므로
탐색 공간이 넓어지고, 더 높은 확률을 갖는 문장을 찾을 가능성이 커진다.

In [ ]:
import torch.nn.functional as F#수학적개산을 할때 쓰는 함수 묶음

def log_probs_from_logits(logits,labels):#파라미터(logits) : 정규화되지 않은 함수 파라미터(lables) :실제로 확률을 알고 싶은 토큰ID
  logp  = F.log_softmax(logits,dim=-1)#
  logp_label = torch.gather(logp,2,labels.unsqueeze(2)).squeeze(-1)#torch.gather()는 특정 위치의 값만 골라오는
                                                                  #logp 모양 [batch,seq_len,vocab_size]
                                                                  '''
                                                                  배치1개,길이3,vocab5라면
                                                                  logp =
                                                                    [
                                                                        [  [-5, -3, -1, -7, -2],   # 첫 토큰의 확률들
                                                                        [-6, -8, -2, -3, -9],   # 두 번째 토큰의 확률들
                                                                        [-4, -1, -8, -2, -5]    # 세 번째 토큰의 확률들
                                                                    ]
                                                                    '''
                                                                    #labels 모양 [batch,seq_len]
                                                                    #예) [[2,3,1]]
                                                                    #labels형태가 [batch,seq_len] 이니깐 [batch,seq_len,1]로 만들어줌
                                                                    #unsqeeze(2) 함수를 사용해서 2번쨰 인덱스에 1을 추가

  return logp_label

이 함수는 하나의 토큰에 대한 로그 확률을 제공하므로,시퀀스의 전체 로그 확률을 얻으려면 각 토큰의 로그 확률을 더해야함

In [ ]:
def sequence_logprob(model,labels,input_len=0):
  with torch.no_grad():
    output =model(labels)
    log_probs = log_probs_from_logits(output.logits[:,:-1,:],labels[:,1:])
    seq_log_prob = torch.sum(log_probs[:,input_len:])
  return seq_log_prob.cpu().numpy()

모델이 입력시퀸스를 생성하지 않았으므로 입력 시퀸스의 로그 확률은 무시합니다.또 로짓과 레이블의 정렬이 중요합니다. 모델은 다음 토큰을 예측하기 때문에 첫 번째 레이블에 대한 로짓을 얻지 못합니다. 또 마지막 로짓에 대한 정답이 없기 때문에 마지막 로짓은 필요하지 않습니다.

In [ ]:
logp = sequence_logprob(model,output_greedy,input_len = len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"\n로그확률 :{logp :.2f}")

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees.


The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.


The researchers were surprised to find that the unicorns were able

로그확률 :-88.66


이를 빔 서치로 생성한 시퀀스와 비교했습니다. generate() 함수에서 빔 서치를 활성화하려면 num_beams 매개변수에 빔 개수를 지정합니다. 빔 크기가 클수록 결과가 더 좋을 가능성이 높습니다. 하지만 각 빔에 대해 병렬적으로 시퀀스를 생성하므로 생성 과정이 훨씬 느림

In [ ]:
output_beam = model.generate(input_ids,max_length = max_length,num_beams = 5,do_sample =False)

logp = sequence_logprob(model,output_beam,input_len = len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\n로그 확률 : {logp : .2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The discovery of the unicorns was made by a team of scientists from the University of California, Santa Cruz, and the National Geographic Society.


The scientists were conducting a study of the Andes Mountains when they discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English

로그 확률 : -56.45


단순한 그리디 서치보다 빔 서치에서 더 높은 로그 확률을 얻었습니다.(높을수록 좋은거) 하지만 빔 서치도 텍스트가 반복되는 문제가 있습니다. 이 문제를 해결하기 위해 no_repeat_ngram-size 매개변수로 n-그램 패널티를 부과하는 방법이 있음
그러면 지금까지 n-그램을 추적해 이전에 보았던 n-그램을 생성하는 경우 다음 토큰 확률이 0이 됩니다.

In [ ]:
output_beam = model.generate(input_ids,max_length = max_length,num_beams = 5,do_sample =False,no_repeat_ngram_size= 2)#n-gram으로 묶은적은 없지만 generate()함수가 자동으로 n-gram으로 묶어줌

logp = sequence_logprob(model,output_beam,input_len = len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\n로그 확률 : {logp : .2f}")

#문장을 다시 바꿔서 n-gram을 테스트 하기 위해서 반복된 문장으로 바꿔서 사용해야함

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The discovery was made by a team of scientists from the University of California, Santa Cruz, and the National Geographic Society.

According to a press release, the scientists were conducting a survey of the area when they came across the herd. They were surprised to find that they were able to converse with the animals in English, even though they had never seen a unicorn in person before. The researchers were

로그 확률 : -94.35


# 샘플링 기반 디코딩 알고리즘

샘플링은 이 확률 분포 전체에서 "가장 높은 단어만 고르는 대신, 확률에 따라 랜덤하게 선택하는 방법

즉, 확률이 높을수록 선택될 가능성이 높고, 하지만 낮은 확률의 단어도 운이 좋으면 뽑힐수 있음

이 덕분에 매번 다른 문장을 만들수 있고,창의적인 문장,다양성이 생겨

단점은 확률이 너무 퍼져 있으면 엉뚱한 문장도 생김

여기서 temperatuere는 확률 분포를 얼마나 "날카롭게","퍼지게"만들지 조절하는거임
즉,모델이 얼마나 확신 있게, 또 자유롭게 단어를 고를지를 조절함

= 0.7	확률이 높은 단어 쪽으로 집중됨	보수적, 논리적 문장

T = 1.0	기본값 (적당히 다양함)	자연스러운 문장

T = 1.5	확률이 퍼짐	창의적이지만 위험

T → 0	사실상 그리디 서치(가장 높은 확률만 선택) 예측은 안정적,다양성 없음

In [ ]:
output_temp = model.generate(input_ids,max_length=max_length,do_sample =True,temperature=2.0,top_k=0)
print(tokenizer.decode(output_temp[0]))

탑-k 샘플링은 확률이 가장 높은 k개 토큰에서만 샘플링해서 확률이 낮은 토큰을 피합니다. 이렇게 하면 확률분포의 롱테일을 잘라내고 확률이 가장 높은 토큰에서만 샘플링하는것이 가능해집니다.

In [ ]:
#근데 이 k값은 수동으로 파라미터로 지정을 해야되는데 어떤 상황 어떤 파라미트러를 설정해줘야 좋은걸까??
#문장생성중 매 타임스텝마다 확률 분포가 달라도 항상 똑같은 k값을 적용해야한다.

output_temp = model.generate(input_ids,max_length=max_length,do_sample =True,temperature=0.5,top_k=0)
print(tokenizer.decode(output_temp[0]))

# 뉴클리어스 샘플링 or  Top-p 샘플링 알고리즘
위에 주석에 단 문제점을 해결하기 위해 나타난게 Top-p 샘플링 기법이다

뉴클리어스 샘플링 또는 탑-p 샘플링에서는 고정된 컷오프 값을 선택하지 않고 어디서 컷오프를 할지 조건을 지정합니다. 이조건은 선택한 특정 확률 질량에 도달할 때입니다. 이 값을 95%로 지정 했다고 생각하면,그 다음 확률에 내림차순으로 모든 토큰을 정렬하고 선택한 토큰의 확률 값이 95%에 도달할 때까지 이 리스트의 맨 위부터 토큰을 하나씩 추가합니다.p값은 누적 확률 그래프의 수평선에 해당함
이 수평선의 아래에 있는 토큰에서만 샘플링을 하게 된다. 출력 분포에 따라 하나의 토큰이 될수도 있고 백개의 토큰이 될수도 있다

izza	0.45	0.45

sushi	0.25	0.70

apple	0.15	0.85

banana	0.10	0.95 <- 여기서 stop!

In [ ]:
#쉽게 말하면 확률이 높은 단어들부터 통에 차곡차곡 넣어놨다가 전체 확률의 합이 95%에 도달했을때 그순간 멈춘다
#그리고 그 통 안에 있는 단어들만 랜덤하게 뽑는다
#즉, 확률이 높은 단어가 몇 개밖에 없다면 후보는 적음
#   확률이 고르게 퍼져 있다면 -> 후보는 많음
output_topp = model.generate(input_ids,max_length=max_length,do_sample =True,top_p=0.90)
print(tokenizer.decode(output_topp[0]))